<a href="https://colab.research.google.com/github/rams1987/Essay_Scoring/blob/main/essay_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: install kaggle

!pip install kaggle


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ramasamyseenivasagan","key":"f41bb0714d3baef12d51ba89162604ef"}'}

In [3]:
# prompt: create a kaggle folder

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle competitions download -c learning-agency-lab-automated-essay-scoring-2

  0% 0.00/11.9M [00:00<?, ?B/s]
100% 11.9M/11.9M [00:00<00:00, 127MB/s]


In [5]:
# prompt: unzip the downloaded file

!unzip learning-agency-lab-automated-essay-scoring-2.zip


Archive:  learning-agency-lab-automated-essay-scoring-2.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [7]:
import os
import numpy as np
import pandas as pd
import spacy

In [8]:
essay_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

essay_df.head()

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [31]:
from textblob import TextBlob
nlp = spacy.load('en_core_web_sm')

In [32]:
def feature_engineering(input_df):
    input_df["sentence_count"] = input_df["full_text"].apply(lambda text: len(list(nlp(text).sents)))
    input_df["word_count"] = input_df["full_text"].apply(lambda text: len(text.split()))
    input_df['average_sentence_length'] = input_df['word_count'] / input_df['sentence_count']
    input_df['unique_words'] = input_df['full_text'].apply(lambda x: len(set(x.split())))
    input_df['sentiment'] = input_df['full_text'].apply(lambda x: TextBlob(x).sentiment.polarity)

    return input_df


In [33]:
input_df = feature_engineering(essay_df)
input_df.head()

,essay_id,full_text,score,sentence_count,word_count,average_sentence_length,unique_words,sentiment
0,000d118,Many people have car where they live. The thin...,3,13,498,38.307692,253,0.179020
1,000fe60,I am a scientist at NASA that is discussing th...,3,20,332,16.600000,177,0.101786
2,001ab80,People always wish they had the same technolog...,4,25,550,22.000000,253,0.168435
3,001bdc0,"We all heard about Venus, the planet without a...",4,21,451,21.476190,251,0.094291
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3,16,373,23.312500,162,0.170952


In [34]:
# prompt: train a model to predict score of the essay

from sklearn.linear_model import LinearRegression

X = input_df[['sentence_count', 'word_count', 'average_sentence_length', 'unique_words', 'sentiment']]
y = input_df['score']

model = LinearRegression().fit(X, y)


In [35]:
test_df = feature_engineering(test_df)

X_test = test_df[['sentence_count', 'word_count', 'average_sentence_length', 'unique_words', 'sentiment']]
y_pred = model.predict(X_test)

print(y_pred)

[3.39993672 2.816332   3.80876369]
